In [344]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import geopandas as gpd
import re

In [345]:
df = pd.read_excel("/Users/santi/Desktop/Work/Bases/base_2012_2022_limpia.xlsx")

df["TIPO_OBRA"] = df["TIPO_OBRA"].astype(str)

df = df[df["TIPO_OBRA"]  == "1"]
df["SUP_CONST"] = df["SUP_CONST"].astype(str).str.replace(",", ".")
df['SUP_CONST'] = df["SUP_CONST"].astype(float)
df["BARRIOS"] = df["BARRIOS"].str.replace(",", ".")
df["BARRIOS"] = df["BARRIOS"].str.replace("NUNEZ", "NUÑEZ")
#df["SUP_CONST"] = df["SUP_CONST"].astype(str).str.replace("321254.00", "321.254")
#df.at[2371, 'SUP_CONST'] = str(df.at[2371, 'SUP_CONST']).replace('321254.00', '321.254')
df.at[2371, 'SUP_CONST'] = 321.254

df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,ANIO,MES,TRIMESTRE,FECHA,DIRECCION,TIPO_OBRA,TIPO_OBRA_CUR,...,ALT_PISOS,BARRIOS,COMUNA,PROPIETARIO,Latitud,Longitud,OBS,add,location,address
0,0,0.0,7,2022,1,1.0,3 ene 2022,GUARDIA VIEJA 3764,1,OBRA MAYOR,...,10,ALMAGRO,5.0,PRIVADO PARTICULAR,-34.601221,-58.417228,NaN,"GUARDIA VIEJA 3764 ALMAGRO Buenos Aires, Argen...",0.0,"GUARDIA VIEJA 3764 ALMAGRO Buenos Aires, Argen..."
1,1,1.0,8,2022,2,1.0,25 feb 2022,YATAY 641,1,OBRA MAYOR,...,11,ALMAGRO,5.0,PERSONERIA JURIDICA,NaN,NaN,NaN,"YATAY 641 ALMAGRO Buenos Aires, Argentina",NaN,"YATAY 641 ALMAGRO Buenos Aires, Argentina"
2,2,2.0,9,2022,3,1.0,3 mar 2022,RAWSON 276,1,Obra Mayor,...,11,ALMAGRO,5.0,PRIVADO PARTICULAR,-34.609157,-58.426283,NaN,"RAWSON 276 ALMAGRO Buenos Aires, Argentina",2.0,"RAWSON 276 ALMAGRO Buenos Aires, Argentina"
3,3,3.0,10,2022,1,1.0,25 ene 2022,TREINTA Y TRES ORIENTALES 682,1,OBRA MEDIA,...,11,ALMAGRO,5.0,PRIVADO PARTICULAR,NaN,NaN,NaN,TREINTA Y TRES ORIENTALES 682 ALMAGRO Buenos A...,NaN,TREINTA Y TRES ORIENTALES 682 ALMAGRO Buenos A...
4,4,4.0,11,2022,3,1.0,2 mar 2022,LAMBARE 847,1,OBRA MAYOR,...,10,ALMAGRO,5.0,PERSONERIA JURIDICA,-34.605103,-58.430465,NaN,"LAMBARE 847 ALMAGRO Buenos Aires, Argentina",4.0,"LAMBARE 847 ALMAGRO Buenos Aires, Argentina"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25515,21592,NaN,17835,2014,10,4.0,2014-10-02 00:00:00,ZUVIRIA 313,1,SIN DATOS,...,NaN,PARQUE CHACABUCO,7.0,NaN,-34.629211,-58.430486,NaN,NaN,NaN,NaN
25517,21594,NaN,17837,2021,1,1.0,18/10/2020,ZUVIRIA 4170,1,Obra Media,...,4,PARQUE AVELLANEDA,9.0,PRIVADO,-34.657364,-58.471501,NaN,NaN,NaN,NaN
25518,21595,NaN,17838,2013,6,2.0,2013-06-03 00:00:00,ZUVIRIA 51,1,SIN DATOS,...,NaN,PARQUE CHACABUCO,7.0,NaN,-34.628684,-58.426622,NaN,NaN,NaN,NaN
25522,21599,NaN,17842,2012,8,3.0,2012-08-24 00:00:00,ZUVIRIA 717,1,SIN DATOS,...,0,PARQUE CHACABUCO,7.0,NaN,-34.629947,-58.435159,NaN,NaN,NaN,NaN


In [346]:
merge = df.groupby(["BARRIOS",'ANIO']).agg(permisadas=("ANIO","size"),SUP_CONST=("SUP_CONST","sum"))
merge

permisadas  SUP_CONST
BARRIOS       ANIO                       
AGRONOMIA     2012           8    5905.91
              2013           6    2156.21
              2014           3    5551.22
              2015           3    2972.95
              2016          10    9653.52
...                        ...        ...
VILLA URQUIZA 2018         110  207948.88
              2019          83  128606.66
              2020          10   14376.06
              2021          48   95615.00
              2022          66   80030.29

[518 rows x 2 columns]

In [347]:
def zscore(arr):
    return (arr - np.mean(arr)) / np.std(arr)

merge['Z_score_permisadas'] = merge.groupby(level=0)['permisadas'].transform(zscore)

merge['Z_score_SUP_CONST'] = merge.groupby(level=0)['SUP_CONST'].transform(zscore)

In [348]:
pca = PCA(n_components=1)
components = pca.fit_transform(merge[["Z_score_permisadas","Z_score_SUP_CONST"]])
merge["IND_actividad"] = components
merge["IND_actividad"].array

#pd.to_excel("Indice_actividad.xlsx")

<PandasArray>
[  0.7637868145131296, -0.40696313632172615, -0.49412075629470936,
  -0.9712003850970822,   1.9341500345462466,  -0.2567746566381655,
   1.1435760556900512, -0.12096107015442402,  -1.6382902836511157,
  -1.9499357172891179,
 ...
   1.2851436209329836, 0.012543931683515187,  -0.8471257924192537,
  0.09792496052456749,  0.20522442230225707,     2.25988023138781,
  0.40632631942207265,  -3.1465072174752264,  -0.9803598496634532,
    -0.74065703583375]
Length: 518, dtype: float64

In [349]:
mask = merge.index.get_level_values(1) == 2022
merge[mask].sort_values(by="IND_actividad",ascending=False)

,,permisadas,SUP_CONST,Z_score_permisadas,Z_score_SUP_CONST,IND_actividad
BARRIOS,ANIO,,,,,
NUÑEZ,2022,49,400002.080,1.842129,2.538898,3.097854
AGRONOMIA,2022,7,13857.770,0.521168,2.302639,1.996733
VERSALLES,2022,11,9007.670,1.835846,0.704835,1.796533
BELGRANO,2022,63,178180.800,1.209007,0.903590,1.493832
VILLA RIACHUELO,2022,6,10891.620,0.312348,1.675451,1.405586
VILLA REAL,2022,6,3810.430,1.185854,0.709637,1.340314
CONSTITUCION,2022,8,26335.740,0.697290,0.666399,0.964274
VILLA GRAL. MITRE,2022,19,12877.690,1.359742,-0.207602,0.814687
SAAVEDRA,2022,56,63730.300,0.976684,-0.000257,0.690439


In [350]:
def linear_regression(group):
    reg = LinearRegression()
    X = group.index.get_level_values(1).values.reshape(-1, 1)  # Access the independent variable from the index
    y = group["IND_actividad"]
    reg.fit(X, y)
    group['pendiente'] = reg.coef_[0]
    #group["error"] = reg.error
    #y_pred = reg.predict(X)
    #error = mean_squared_error(y, y_pred)
    return group



result = merge.groupby(level=0).apply(linear_regression)
#result.groupby(level=0)["pendiente"].mean().sort_values(ascending=False)
#result.to_excel("Indice_actividad.xlsx")

/var/folders/d3/r414kbwd72g3nnhvylz22y9m0000gp/T/ipykernel_29038/2833327811.py:14: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  result = merge.groupby(level=0).apply(linear_regression)


In [351]:
gdf = gpd.read_file("https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-educacion/barrios/barrios.geojson")
result_copy = result.reset_index()
merged_gdf = gpd.GeoDataFrame(pd.merge(result_copy, gdf, left_on='BARRIOS',right_on="BARRIO"))

merged_gdf

merged_gdf.to_file(r"/Users/santi/Desktop/Shape/gdf.shp")

/var/folders/d3/r414kbwd72g3nnhvylz22y9m0000gp/T/ipykernel_29038/1304848073.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_gdf.to_file(r"/Users/santi/Desktop/Shape/gdf.shp")


In [352]:
gdf = gpd.read_file("https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ministerio-de-educacion/barrios/barrios.geojson")
gdf
result = result.reset_index().merge(gdf, left_on='BARRIOS', right_on='BARRIO')
#result.set_index(['BARRIOS', 'ANIO'], inplace=True)
result = result.iloc[:, :10]
result.drop(columns="BARRIO",inplace=True)
result

,BARRIOS,ANIO,permisadas,SUP_CONST,Z_score_permisadas,Z_score_SUP_CONST,IND_actividad,pendiente,COMUNA
0,AGRONOMIA,2012,8,5905.91,0.858395,0.221763,0.763787,-0.022998,15.0
1,AGRONOMIA,2013,6,2156.21,0.183942,-0.759475,-0.406963,-0.022998,15.0
2,AGRONOMIA,2014,3,5551.22,-0.827738,0.128946,-0.494121,-0.022998,15.0
3,AGRONOMIA,2015,3,2972.95,-0.827738,-0.545747,-0.971200,-0.022998,15.0
4,AGRONOMIA,2016,10,9653.52,1.532848,1.202453,1.934150,-0.022998,15.0
...,...,...,...,...,...,...,...,...,...
513,VILLA URQUIZA,2018,110,207948.88,1.359079,1.836874,2.259880,-0.225560,12.0
514,VILLA URQUIZA,2019,83,128606.66,0.369749,0.204883,0.406326,-0.225560,12.0
515,VILLA URQUIZA,2020,10,14376.06,-2.305105,-2.144728,-3.146507,-0.225560,12.0
516,VILLA URQUIZA,2021,48,95615.00,-0.912715,-0.473723,-0.980360,-0.225560,12.0


In [353]:
result.to_excel("/Users/santi/Desktop/Work/Bases/barrios_relativo.xlsx")

In [354]:
barrios = result.reset_index()
barrios = barrios[barrios["ANIO"] == 2022]

barrios["densidad"] = barrios["SUP_CONST"]/(barrios["PERIMETRO"]/100)


barrios = barrios[["BARRIOS","permisadas","SUP_CONST","geometry","densidad"]]
barrios["tamaño"] = barrios["SUP_CONST"]/(barrios["permisadas"])


barrios["Z_permisadas"] = zscore(barrios["permisadas"])
barrios["Z_densidad"] = zscore(barrios["densidad"])

pca = PCA(n_components=1)
components = pca.fit_transform(barrios[["Z_permisadas","Z_densidad"]])
barrios["IND_actividad"] = components

barrios['quantiles_tamaño'] = pd.qcut(barrios['tamaño'], q=5, labels=['1', '2', '3', '4',"5"])
barrios['quartile_range_tamaño'] = pd.qcut(barrios['tamaño'], q=5,precision=0)

barrios['quantiles_densidad'] = pd.qcut(barrios['densidad'], q=5, labels=['1', '2', '3', '4',"5"])
barrios['quartile_range_densidad'] = pd.qcut(barrios['densidad'], q=5,precision=0)


barrios['quantiles_permisos'] = pd.qcut(barrios['permisadas'], q=5, labels=['1', '2', '3', '4',"5"])
barrios['quartile_range_permisadas'] = pd.qcut(barrios['permisadas'], q=5,precision=0)



barrios['quantiles_sup'] = pd.qcut(barrios['SUP_CONST'], q=5, labels=['1', '2', '3', '4',"5"])
barrios['quartile_range_sup'] = pd.qcut(barrios['SUP_CONST'], q=5,precision=0)



barrios['quantiles_ind'] = pd.qcut(barrios['IND_actividad'], q=5, labels=['1', '2', '3', '4',"5"])
barrios['quartile_range_ind'] = pd.qcut(barrios['IND_actividad'], q=5,precision=2)





barrios.drop(columns=["Z_densidad","Z_permisadas"],inplace=True)

barrios.reset_index(inplace=True,drop=True)

columns_to_apply = [ "quartile_range_sup", "quartile_range_permisadas","quartile_range_densidad","quartile_range_tamaño"]


barrios[columns_to_apply] = barrios[columns_to_apply].applymap(lambda x: re.sub(r"\.0", "", str(x)).replace("]", ")"))
barrios["quartile_range_ind"] = barrios["quartile_range_ind"].astype(str).str.replace("]", ")", regex=True)


barrios["quartile_range_ind"] = barrios["quantiles_ind"].astype(str) + ": " + barrios["quartile_range_ind"].astype(str)
barrios["quartile_range_sup"] = barrios["quantiles_sup"].astype(str) + ": " + barrios["quartile_range_sup"].astype(str)
barrios["quartile_range_permisadas"] = barrios["quantiles_permisos"].astype(str) + ": " + barrios["quartile_range_permisadas"].astype(str)
barrios["quartile_range_densidad"] = barrios["quantiles_densidad"].astype(str) + ": " + barrios["quartile_range_densidad"].astype(str)
barrios["quartile_range_tamaño"] = barrios["quantiles_tamaño"].astype(str) + ": " + barrios["quartile_range_tamaño"].astype(str)



barrios
#barrios.to_excel("/Users/santi/Desktop/Work/Bases/Barrios_quartilico_obras_nuevas.xlsx")

KeyError: 'PERIMETRO'